In [1]:
"""
Description: Script to optimize Duca's currency mix.
Author: Jeroen van Dijk & Victor de Graaff
Date: 04-11-2020
Maintainer: Jeroen van Dijk & Victor de Graaff
Email: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl
Status: Dev
"""

"\nDescription: Script to optimize Duca's currency mix.\nAuthor: Jeroen van Dijk & Victor de Graaff\nDate: 04-11-2020\nMaintainer: Jeroen van Dijk & Victor de Graaff\nEmail: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl\nStatus: Dev\n"

In [2]:
# Load common imports
%run ./CommonImports.ipynb

In [3]:
# Load common functions and currencies lists:
# - all_currencies
# - obsolete_currencies
# - p13_currencies
# - f_currencies
# - ff_currencies
# - currencies_per_continent
%run ./Utilities.ipynb

In [4]:
# Load loss functions
# - calculate_loss_function_around_one(weights)
# - calculate_loss_function_vs_t_minus_one(weights)
# - calculate_loss_function_vs_t_minus_one_with_balancing(weights)
# - calculate_loss_function_vs_t_minus_one_for_period(weights, max_date)
%run ./LossFunctions.ipynb

In [6]:
from scipy.stats import pearsonr

currency_pairs = [(c1, c2) for c1 in all_currencies for c2 in all_currencies if c1 > c2]

highly_correlated_currencies = set()
highly_unstable_currencies = set()

for base_currency in all_currencies:
    print(f"Evaluating correlations using base currency {base_currency}")
    df = create_original_df(base_currency)
    
    median = np.median(np.array([pearsonr(df[c1], df[c2])[0] 
                                 for c1, c2 in currency_pairs 
                                 if c1 != base_currency and c2 != base_currency]))
    
    if median > .9:
        highly_unstable_currencies.add(base_currency)
        continue

    for c1, c2 in currency_pairs:
        if base_currency == c1 or base_currency == c2:
            continue
            
        if c1 in highly_correlated_currencies or c2 in highly_correlated_currencies:
            continue
            
        corr = pearsonr(df[c1], df[c2])[0]

        if abs(corr) > .98:
            if trading_volumes[c1] >= trading_volumes[c2] and \
               trading_volumes[base_currency] >= trading_volumes[c2]:
                print(f"According to {base_currency} {c1} and {c2} are highly corrolated:", corr)
                highly_correlated_currencies.add(c2)
            elif trading_volumes[c2] >= trading_volumes[c1] and \
                 trading_volumes[base_currency] >= trading_volumes[c1]:
                print(f"According to {base_currency} {c2} and {c1} are highly corrolated:", corr)
                highly_correlated_currencies.add(c1)

    print(highly_unstable_currencies, highly_correlated_currencies)
    
highly_unstable_currencies, highly_correlated_currencies

Evaluating correlations using base currency KRW
According to KRW EUR and DKK are highly corrolated: 0.9999186626852319
According to KRW EUR and HRK are highly corrolated: 0.9935821354027969
set() {'HRK', 'DKK'}
Evaluating correlations using base currency SGD
set() {'HRK', 'DKK'}
Evaluating correlations using base currency NOK
set() {'HRK', 'DKK'}
Evaluating correlations using base currency JPY
According to JPY USD and HKD are highly corrolated: 0.9997071174459964
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency AUD
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency HUF
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency GBP
set() {'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency THB
According to THB HUF and RON are highly corrolated: 0.9816769330765294
set() {'RON', 'HKD', 'HRK', 'DKK'}
Evaluating correlations using base currency HKD
set() {'RON', 'HKD', 'HRK', 'DKK'}
Evaluating correlations using base curr

({'TRY', 'ZAR'}, {'DKK', 'HKD', 'HRK', 'MYR', 'PHP', 'RON', 'RUB', 'THB'})

In [8]:
# Create dataframe with pre-defined base currency
df = create_original_df("EUR")

In [35]:
# Determine splits for dev- (train and test) and validation set
train_start = date(2005, 4, 1)
test_start = date(2018, 1, 1)
validation_start = date(2018, 1, 1)
validation_end = date(2020, 10, 1)

# Create dev (train and test) and validation set
train, test, dev, val = split_data(df, train_start, test_start, validation_start, validation_end)

In [10]:
# Create exchange table for each currency
exchange_table = dict()
for base_currency in all_currencies:
    exchange_table[base_currency] = 1/train[all_currencies].divide(train[base_currency], axis=0)

# Preview USD exchange table
exchange_table["USD"]

,SGD,HRK,AUD,JPY,TRY,HUF,PHP,THB,USD,IDR,HKD,SEK,CZK,PLN,ZAR,RUB,DKK,MYR,RON,CNY,CAD,NOK,KRW,CHF,GBP,NZD,EUR
date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-04-01,0.60418,0.17460,0.77123,0.00932,0.73963,0.00524,0.01827,0.02554,1.00000,0.00011,0.12823,0.14134,0.04315,0.31703,0.16175,0.03588,0.17394,0.26316,0.35967,0.12082,0.82353,0.15813,0.00099,0.83461,1.88728,0.71051,1.29590
2005-04-02,0.60293,0.17424,0.77012,0.00930,0.73758,0.00523,0.01826,0.02546,1.00000,0.00011,0.12822,0.14100,0.04307,0.31580,0.16153,0.03587,0.17360,0.26316,0.35897,0.12082,0.82270,0.15780,0.00099,0.83284,1.88327,0.70945,1.29337
2005-04-03,0.60168,0.17387,0.76901,0.00928,0.73553,0.00522,0.01825,0.02537,1.00000,0.00011,0.12822,0.14065,0.04299,0.31458,0.16132,0.03586,0.17326,0.26317,0.35827,0.12082,0.82187,0.15748,0.00099,0.83106,1.87926,0.70838,1.29083
2005-04-04,0.60044,0.17351,0.76790,0.00926,0.73349,0.00521,0.01824,0.02529,1.00000,0.00011,0.12821,0.14031,0.04291,0.31336,0.16110,0.03585,0.17292,0.26317,0.35756,0.12082,0.82104,0.15715,0.00099,0.82929,1.87525,0.70731,1.28830
2005-04-05,0.60059,0.17266,0.76441,0.00921,0.73137,0.00518,0.01824,0.02522,1.00000,0.00011,0.12821,0.13960,0.04271,0.31019,0.16014,0.03579,0.17195,0.26316,0.35554,0.12082,0.81681,0.15691,0.00098,0.82427,1.87253,0.70431,1.28100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-27,0.74638,0.15776,0.77664,0.00883,0.26157,0.00382,0.02002,0.03048,1.00000,0.00007,0.12797,0.12048,0.04603,0.28400,0.08031,0.01734,0.15977,0.24535,0.25581,0.15255,0.79152,0.12063,0.00093,1.01045,1.34266,0.70698,1.18950
2017-12-28,0.74737,0.15888,0.77847,0.00886,0.26252,0.00385,0.02004,0.03066,1.00000,0.00007,0.12795,0.12122,0.04654,0.28545,0.08100,0.01735,0.16028,0.24591,0.25653,0.15300,0.79301,0.12095,0.00093,1.01965,1.34440,0.70800,1.19340
2017-12-29,0.74844,0.16120,0.78151,0.00888,0.26379,0.00386,0.02006,0.03066,1.00000,0.00007,0.12797,0.12183,0.04697,0.28712,0.08100,0.01728,0.16109,0.24709,0.25744,0.15367,0.79746,0.12188,0.00094,1.02487,1.35174,0.71175,1.19930


In [11]:
def run_optimization(selected_currencies, 
                     max_iter=100, 
                     loss_function=calculate_loss_function_around_one,
                     reference_currencies=all_currencies):
    starting_point = np.ones(len(selected_currencies))/len(selected_currencies)
    bounds = [(0, 1) for _ in range(len(starting_point))]
    
    result = minimize(fun=loss_function, 
                      x0=starting_point, 
                      args=(selected_currencies, reference_currencies),
                      bounds=bounds,
                      options={"disp": True, 
                               "maxiter": max_iter})
    
    output = pd.Series(result.x/result.x.sum(), index=reference_date_exhange_rate.loc[selected_currencies].index)
    output = output.sort_values(ascending=False)
        
    return output

In [12]:
def try_currencies(currencies_to_try, currencies, reference_currencies):
    for currency in currencies_to_try:
        if currency in currencies:
            return None, 1

    new_mix = run_optimization(selected_currencies=currencies + currencies_to_try, 
                               max_iter=100, 
                               loss_function=calculate_loss_function_around_one,
                               reference_currencies=reference_currencies)

    new_score = calculate_loss_function_around_one(new_mix, currencies + currencies_to_try, reference_currencies)

    return new_mix, new_score

In [ ]:
candidate_currencies = [c for c in all_currencies if c not in highly_correlated_currencies and
                                                     c not in highly_unstable_currencies]
more_stable_currencies = [c for c in all_currencies if c not in highly_unstable_currencies]
candidate_currencies.sort()

reference_date_exhange_rate = train[more_stable_currencies].iloc[0]

with open("logs/output-loss-function-on-more-stable-currencies-0_001-0_995.log", "a") as output:
    for currency in candidate_currencies:
        selected_currencies_so_far = [currency]
        best_score = 1
        last_score = 1

        while best_score == 1 or best_candidate is not None:
            output.write(f"Attempting to improve set, starting from: {selected_currencies_so_far}\n")
            print(f"Attempting to improve set, starting from: {selected_currencies_so_far}")
            best_candidate = None

            for candidate_currency in candidate_currencies:
                new_mix, new_score = try_currencies([candidate_currency], 
                                                    selected_currencies_so_far, 
                                                    more_stable_currencies)

                if (new_score < .995 * last_score) and (new_score < best_score):
                    best_score = new_score
                    best_candidate = candidate_currency

            last_score = best_score

            if best_candidate is not None:
                selected_currencies_so_far += [best_candidate]
                
            if len(selected_currencies_so_far) == 7:
                # Let's make sure this process ever finishes
                break

        output.write(f"Best set: {selected_currencies_so_far} ({best_score})\n")
        print(f"Best set: {selected_currencies_so_far} ({best_score})")

Attempting to improve set, starting from: ['AUD']
Attempting to improve set, starting from: ['AUD', 'USD']
Attempting to improve set, starting from: ['AUD', 'USD', 'EUR']
Attempting to improve set, starting from: ['AUD', 'USD', 'EUR', 'GBP']
Attempting to improve set, starting from: ['AUD', 'USD', 'EUR', 'GBP', 'JPY']
Attempting to improve set, starting from: ['AUD', 'USD', 'EUR', 'GBP', 'JPY', 'CNY']
Best set: ['AUD', 'USD', 'EUR', 'GBP', 'JPY', 'CNY'] (0.010290543996274003)
Attempting to improve set, starting from: ['CAD']
Attempting to improve set, starting from: ['CAD', 'USD']
Attempting to improve set, starting from: ['CAD', 'USD', 'EUR']
Attempting to improve set, starting from: ['CAD', 'USD', 'EUR', 'SEK']
Attempting to improve set, starting from: ['CAD', 'USD', 'EUR', 'SEK', 'JPY']
Best set: ['CAD', 'USD', 'EUR', 'SEK', 'JPY'] (0.010391696436945406)
Attempting to improve set, starting from: ['CHF']
Attempting to improve set, starting from: ['CHF', 'GBP']
Attempting to improve s

In [14]:
selected_currencies = ['JPY', 'USD', 'EUR', 'NZD', 'KRW', 'CNY', 'GBP']
new_mix, new_score = try_currencies([], 
                                    selected_currencies, 
                                    more_stable_currencies)

In [15]:
(new_mix * 100).map(lambda x: round(x, 2)), new_score

(USD   28.88000
 EUR   20.06000
 JPY   14.83000
 GBP   12.48000
 KRW   10.85000
 CNY   10.51000
 NZD    2.39000
 dtype: float64,
 0.010239016950012721)

In [37]:
duca_mix = new_mix

duca_mix.index.name = "currency"
duca_mix.name = "weight"
duca_mix = duca_mix.map(lambda x: round(x, 4))
duca_mix.to_csv("duca_mix.csv", sep=";")

In [27]:
calculate_loss_function_around_one?

Signature:
calculate_loss_function_around_one(
    weights,
    selected_currencies,
    reference_currencies,
)
Docstring: <no docstring>
File:      /opt/app/data/CurrencyBucket/<ipython-input-12-efbd4238dfcb>
Type:      function


### Comparison and validation

#### Development set

In [31]:
libra_mix = get_libra_mix(all_currencies)

calculate_loss_function_around_one(libra_mix,
                                   selected_currencies=libra_mix.index,
                                   reference_currencies=more_stable_currencies)

0.010396552497994496

In [33]:
sdr_mix = get_sdr_mix(all_currencies)

calculate_loss_function_around_one(sdr_mix,
                                   selected_currencies=sdr_mix.index,
                                   reference_currencies=more_stable_currencies)

0.01048454497936993

#### Validation set

In [36]:
# Create exchange table for each currency
exchange_table = dict()
for base_currency in all_currencies:
    exchange_table[base_currency] = 1/val[all_currencies].divide(val[base_currency], axis=0)

# Preview USD exchange table
exchange_table["USD"]

,SGD,HRK,AUD,JPY,TRY,HUF,PHP,THB,USD,IDR,HKD,SEK,CZK,PLN,ZAR,RUB,DKK,MYR,RON,CNY,CAD,NOK,KRW,CHF,GBP,NZD,EUR
date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.75156,0.16153,0.78246,0.00891,0.26552,0.00390,0.02006,0.03080,1.00000,0.00007,0.12797,0.12253,0.04724,0.28912,0.08098,0.01741,0.16184,0.24836,0.25885,0.15393,0.79751,0.12304,0.00094,1.02843,1.35519,0.71163,1.20470
2018-01-02,0.75260,0.16164,0.78278,0.00891,0.26610,0.00391,0.02006,0.03084,1.00000,0.00007,0.12797,0.12276,0.04732,0.28979,0.08097,0.01746,0.16208,0.24879,0.25932,0.15401,0.79753,0.12343,0.00094,1.02961,1.35633,0.71159,1.20650
2018-01-03,0.75200,0.16158,0.78382,0.00891,0.26539,0.00389,0.02004,0.03074,1.00000,0.00007,0.12792,0.12237,0.04707,0.28865,0.08078,0.01740,0.16151,0.24907,0.25937,0.15381,0.79903,0.12339,0.00094,1.02445,1.35639,0.70966,1.20230
2018-01-04,0.75204,0.16226,0.78354,0.00888,0.26594,0.00391,0.02009,0.03104,1.00000,0.00007,0.12791,0.12284,0.04729,0.29034,0.08126,0.01750,0.16206,0.24925,0.26064,0.15393,0.79827,0.12355,0.00094,1.02567,1.35405,0.71151,1.20650
2018-01-05,0.75314,0.16200,0.78413,0.00883,0.26691,0.00390,0.02005,0.03107,1.00000,0.00007,0.12788,0.12251,0.04706,0.28986,0.08090,0.01751,0.16177,0.25000,0.25986,0.15412,0.79938,0.12364,0.00094,1.02450,1.35515,0.71539,1.20450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-26,0.72678,0.15427,0.70427,0.00948,0.13013,0.00320,0.02062,0.03161,1.00000,0.00007,0.12903,0.10972,0.04295,0.25574,0.05853,0.01282,0.15640,0.23980,0.23889,0.14654,0.74750,0.10477,0.00085,1.07793,1.27887,0.65546,1.16460
2020-09-27,0.72744,0.15442,0.70549,0.00948,0.12912,0.00320,0.02061,0.03159,1.00000,0.00007,0.12903,0.10998,0.04298,0.25611,0.05860,0.01276,0.15657,0.23964,0.23916,0.14666,0.74731,0.10515,0.00085,1.07845,1.28411,0.65571,1.16580
2020-09-28,0.72810,0.15457,0.70672,0.00949,0.12814,0.00320,0.02061,0.03157,1.00000,0.00007,0.12903,0.11024,0.04301,0.25647,0.05867,0.01271,0.15673,0.23949,0.23942,0.14677,0.74712,0.10553,0.00086,1.07896,1.28939,0.65595,1.16700


In [38]:
calculate_loss_function_around_one(duca_mix,
                                   selected_currencies=duca_mix.index,
                                   reference_currencies=more_stable_currencies)

0.01874640510450694

In [39]:
calculate_loss_function_around_one(libra_mix,
                                   selected_currencies=libra_mix.index,
                                   reference_currencies=more_stable_currencies)

0.019043172930094672

In [40]:
calculate_loss_function_around_one(sdr_mix,
                                   selected_currencies=sdr_mix.index,
                                   reference_currencies=more_stable_currencies)

0.01910046574220742